In [2]:
# Get pandas and postgres to work together
import psycopg2 as pg
import pandas as pd

# We are also going to do some basic viz
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline

plt.rcParams['figure.figsize'] = (9, 6)
sns.set(context='notebook', style='whitegrid', font_scale=1.2)

In [53]:
# Postgres info to connect
connection_args = {
    'host': 'localhost',  # We are connecting to our _local_ version of psql
    'dbname': 'myersbriggs',    # DB that we are connecting to
    'port': 5432          # port we opened on AWS
}

connection = pg.connect(**connection_args)  # What is that "**" there??
cur = connection.cursor()

## Read data from .csv
```python
df = pd.read_csv('mbti_1.csv')
```

## Create a table in PostgreSQL database

```python
sqlCreateTable = "create table raw_data (id bigint, type varchar(4), posts text);"

cur.execute(sqlCreateTable)
connection.commit()
```

In [57]:
cur.close()

## Inserting the data into the database.
```python
def single_insert(conn, insert_req, values):
    """ Execute a single INSERT request """
    cursor = conn.cursor()
    try:
        cursor.execute(insert_req, values)
        conn.commit()
    except (Exception, pg.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

    # Inserting each row
for i in df.index:

    query = "INSERT into raw_data(id, type, posts) values('%s',%s,%s);"
    values = (i, df['type'][i], df['posts'][i])
    single_insert(connection, query, values)
```

In [58]:
# Close the connection
connection.close()

In [56]:
postgreSQL_select_Query = "SELECT * FROM raw_data"

cur.execute(postgreSQL_select_Query)
print("Selecting rows from mobile table using cursor.fetchall")
myers_briggs = cur.fetchall() 

Selecting rows from mobile table using cursor.fetchall
